In [5]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Download NLTK stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prasanna.v\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Load the text file
# Replace 'SMSSpamCollection' with your file path if necessary
with open('SMSSpamCollection.txt', 'r', encoding='latin-1') as file:
    data = file.readlines()

# Split into labels and messages
labels = []
messages = []
for line in data:
    label, message = line.strip().split('\t', 1)
    labels.append(label)
    messages.append(message)

# Create a DataFrame
df = pd.DataFrame({'label': labels, 'message': messages})

# Map labels to binary values: ham = 0, spam = 1
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

df.head()


label                                            message
0      0  Go until jurong point, crazy.. Available only ...
1      0                      Ok lar... Joking wif u oni...
2      1  Free entry in 2 a wkly comp to win FA Cup fina...
3      0  U dun say so early hor... U c already then say...
4      0  Nah I don't think he goes to usf, he lives aro...

In [8]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Apply preprocessing
df['message'] = df['message'].apply(preprocess_text)

df.head()

label                                            message
0      0  go jurong point crazy available bugis n great ...
1      0                            ok lar joking wif u oni
2      1  free entry 2 wkly comp win fa cup final tkts 2...
3      0                u dun say early hor u c already say
4      0        nah dont think goes usf lives around though

In [9]:
# Vectorize the messages
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['message'])
y = df['label']

print(f"Shape of the dataset: {X.shape}")


Shape of the dataset: (5574, 9484)


In [10]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape}")
print(f"Testing set size: {X_test.shape}")


Training set size: (4459, 9484)
Testing set size: (1115, 9484)


In [11]:
# Train the Naive Bayes model
model = MultinomialNB()
model.fit(X_train, y_train)


MultinomialNB()

In [12]:
# Predict and evaluate
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9730941704035875
Confusion Matrix:
 [[934  20]
 [ 10 151]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.98       954
           1       0.88      0.94      0.91       161

    accuracy                           0.97      1115
   macro avg       0.94      0.96      0.95      1115
weighted avg       0.97      0.97      0.97      1115



In [13]:
def predict_message(message):
    message = preprocess_text(message)
    vectorized_message = vectorizer.transform([message])
    prediction = model.predict(vectorized_message)
    return 'Spam' if prediction[0] == 1 else 'Ham'

# Test examples
print(predict_message("Hi, how are you?"))
print(predict_message("Hi, can we meet for lunch tomorrow?"))


Spam
Ham
